<a href="https://colab.research.google.com/github/amikoshimrah/Deployment/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pymysql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
try:
    conn = pymysql.connect(
        host ='127.0.0.1',
        user = 'root',
        password = 'Mysql@2025',
        port = 3306
    )
    query ='select * from titanic.titanic'
    print('connection successfully Established')
    df = pd.read_sql(query,conn)
    display(df.head())
except pymysql.error in e:
    print(f' Sql_error: {e}')
finally:
    if 'conn' in locals():
        conn.close()
        print('Connection successfully closed')


connection successfully Established


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S


Connection successfullt closed


In [ ]:
df.drop(['PassengerId', 'Name'], axis=1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22,1,0,A/5 21171,7.2500,,S
1,1,1,female,38,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26,0,0,STON/O2. 3101282,7.9250,,S
3,1,1,female,35,1,0,113803,53.1000,C123,S
4,0,3,male,35,0,0,373450,8.0500,,S
...,...,...,...,...,...,...,...,...,...,...
709,0,3,female,39,0,5,382652,29.1250,,Q
710,0,2,male,27,0,0,211536,13.0000,,S
711,1,1,female,19,0,0,112053,30.0000,B42,S
712,1,1,male,26,0,0,111369,30.0000,C148,C


In [ ]:
# getting total number of rows and column in the dataframe
print(f' Shape of the dataframe = {df.shape}')
print(f' Total number of rows in the dataset =  {df.shape[0]}')
print(f' Total number of columns in the dataset =  {df.shape[1]}')
print(f' The missing value of NaNs are : {df.isna().sum().sum() }')  #checking missing value NaN
print(f' The Number of duplicate records are : {df.duplicated().sum()}')  # Checcking if there is any duplicate record

 Shape of the dataframe = (714, 12)
 Total number of rows in the dataset =  714
 Total number of columns in the dataset =  12
 The missing value of NaNs are : 0
 The Number of duplicate records are : 0


In [ ]:
#let's import libraries for ML
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 3. Drop Irrelevant Columns
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [ ]:
# 4. Encode Categorical Variables
le_sex = LabelEncoder()
df['Sex'] = le_sex.fit_transform(df['Sex'])  # male=1, female=0
le_embarked = LabelEncoder()
df['Embarked'] = le_embarked.fit_transform(df['Embarked'])  # C=0, Q=1, S=2

In [ ]:
# 5. Split Features & Target
X = df.drop('Survived', axis=1)
y = df['Survived']

In [ ]:
# 6. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 7. Train XGBoost Model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# 8. Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
# 9. Evaluation Function
def evaluate(y_true, y_pred, label="Dataset"):
    print(f"\n📊 Evaluation on {label}")
    print("Accuracy :", round(accuracy_score(y_true, y_pred), 3))
    print("Precision:", round(precision_score(y_true, y_pred), 3))
    print("Recall   :", round(recall_score(y_true, y_pred), 3))
    print("F1 Score :", round(f1_score(y_true, y_pred), 3))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [ ]:
# 10. Evaluate
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_test, y_test_pred, "Testing Set")


📊 Evaluation on Training Set
Accuracy : 0.984
Precision: 0.996
Recall   : 0.966
F1 Score : 0.98
Confusion Matrix:
 [[336   1]
 [  8 226]]

📊 Evaluation on Testing Set
Accuracy : 0.79
Precision: 0.75
Recall   : 0.696
F1 Score : 0.722
Confusion Matrix:
 [[74 13]
 [17 39]]


In [ ]:
import pickle
with open('titanic_classification.pkl','wb') as file:
    pickle.dump(model,file)
print('save as titanic_classification.pkl successfully')

save as titanic_classification.pkl successfully
